# Modelo SVR 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
import math
import warnings

In [ ]:
warnings.filterwarnings("ignore")
style.use('ggplot')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [ ]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.1 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [ ]:
import yfinance as yf
# Dataset de Microsoft Corporation (MSFT)
msft = yf.Ticker('MSFT')
df = msft.history(period="max", auto_adjust=True)
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1986-03-13,0.056,0.064,0.056,0.061,1031788800,0.000,0.000
1986-03-14,0.061,0.064,0.061,0.063,308160000,0.000,0.000
1986-03-17,0.063,0.065,0.063,0.064,133171200,0.000,0.000
1986-03-18,0.064,0.065,0.062,0.063,67766400,0.000,0.000
1986-03-19,0.063,0.063,0.061,0.062,47894400,0.000,0.000
...,...,...,...,...,...,...,...
2022-08-01,277.820,281.280,275.840,278.010,21539600,0.000,0.000
2022-08-02,276.000,277.890,272.380,274.820,22754200,0.000,0.000
2022-08-03,276.760,283.500,276.610,282.470,23518900,0.000,0.000


In [ ]:
#eliminamos las variables sin valores
df = df.drop(['Dividends'], axis='columns')
df = df.drop(['Stock Splits'], axis='columns')
df

,Open,High,Low,Close,Volume
Date,,,,,
1986-03-13,0.056,0.064,0.056,0.061,1031788800
1986-03-14,0.061,0.064,0.061,0.063,308160000
1986-03-17,0.063,0.065,0.063,0.064,133171200
1986-03-18,0.064,0.065,0.062,0.063,67766400
1986-03-19,0.063,0.063,0.061,0.062,47894400
...,...,...,...,...,...
2022-08-01,277.820,281.280,275.840,278.010,21539600
2022-08-02,276.000,277.890,272.380,274.820,22754200
2022-08-03,276.760,283.500,276.610,282.470,23518900


## Variables adicionales:

RSI (Indicador de fuerza relativa):

In [ ]:
delta = df['Close'].diff()
up = delta.clip(lower = 0)
down = -1*delta.clip(upper=0)
ema_up = up.ewm(com=13, adjust=False).mean()
ema_down = down.ewm(com=13, adjust=False).mean()
rs = ema_up/ema_down
df['RSI'] = 100 - (100/(1+rs))
df

,Open,High,Low,Close,Volume,RSI
Date,,,,,,
1986-03-13,0.056,0.064,0.056,0.061,1031788800,NaN
1986-03-14,0.061,0.064,0.061,0.063,308160000,100.000
1986-03-17,0.063,0.065,0.063,0.064,133171200,100.000
1986-03-18,0.064,0.065,0.062,0.063,67766400,94.353
1986-03-19,0.063,0.063,0.061,0.062,47894400,90.678
...,...,...,...,...,...,...
2022-08-01,277.820,281.280,275.840,278.010,21539600,61.220
2022-08-02,276.000,277.890,272.380,274.820,22754200,58.269
2022-08-03,276.760,283.500,276.610,282.470,23518900,62.889


MACD:

In [ ]:
exp1 = df['Close'].ewm(span=12, adjust=False).mean()
exp2 = df['Close'].ewm(span=26, adjust=False).mean()
df['MACD'] = exp1 -exp2
df['Signal line'] = df['MACD'].ewm(span=9, adjust=False).mean()

df.head()

,Open,High,Low,Close,Volume,RSI,MACD,Signal line
Date,,,,,,,,
1986-03-13,0.056,0.064,0.056,0.061,1031788800,NaN,0.000,0.000
1986-03-14,0.061,0.064,0.061,0.063,308160000,100.000,0.000,0.000
1986-03-17,0.063,0.065,0.063,0.064,133171200,100.000,0.000,0.000
1986-03-18,0.064,0.065,0.062,0.063,67766400,94.353,0.000,0.000
1986-03-19,0.063,0.063,0.061,0.062,47894400,90.678,0.000,0.000


Eliminando valores perdidos (faltantes)

In [ ]:
df.isnull().sum()

Open           0
High           0
Low            0
Close          0
Volume         0
RSI            1
MACD           0
Signal line    0
dtype: int64

In [ ]:
#Quitamos las filas que tengan el valor de NaN
df=df.dropna()
df

,Open,High,Low,Close,Volume,RSI,MACD,Signal line
Date,,,,,,,,
1986-03-14,0.061,0.064,0.061,0.063,308160000,100.000,0.000,0.000
1986-03-17,0.063,0.065,0.063,0.064,133171200,100.000,0.000,0.000
1986-03-18,0.064,0.065,0.062,0.063,67766400,94.353,0.000,0.000
1986-03-19,0.063,0.063,0.061,0.062,47894400,90.678,0.000,0.000
1986-03-20,0.062,0.062,0.059,0.060,58435200,85.310,0.000,0.000
...,...,...,...,...,...,...,...,...
2022-08-01,277.820,281.280,275.840,278.010,21539600,61.220,3.386,0.903
2022-08-02,276.000,277.890,272.380,274.820,22754200,58.269,3.732,1.469
2022-08-03,276.760,283.500,276.610,282.470,23518900,62.889,4.572,2.089


Seleccionamos las características para los modelos

In [ ]:
data = df[['Open', 'High', 'Low','Close', 'Volume','RSI', 'MACD']]
data.head()

,Open,High,Low,Close,Volume,RSI,MACD
Date,,,,,,,
1986-03-14,0.061,0.064,0.061,0.063,308160000,100.000,0.000
1986-03-17,0.063,0.065,0.063,0.064,133171200,100.000,0.000
1986-03-18,0.064,0.065,0.062,0.063,67766400,94.353,0.000
1986-03-19,0.063,0.063,0.061,0.062,47894400,90.678,0.000
1986-03-20,0.062,0.062,0.059,0.060,58435200,85.310,0.000


In [ ]:
# Información del dataset
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 9174 entries, 1986-03-14 to 2022-08-05
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    9174 non-null   float64
 1   High    9174 non-null   float64
 2   Low     9174 non-null   float64
 3   Close   9174 non-null   float64
 4   Volume  9174 non-null   int64  
 5   RSI     9174 non-null   float64
 6   MACD    9174 non-null   float64
dtypes: float64(6), int64(1)
memory usage: 573.4 KB


Dividimos los datos en entrenamiento y prueba

In [ ]:
from sklearn.model_selection import train_test_split

# X son nuestras variables independientes
X = data.drop(['Close'],axis = 1)

# y es nuestra variable dependiente
y = data.Close

# División 75% de datos para entrenamiento, 25% de datos para test
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=0)

## Modelo Support Vector Regression (SVR)

**Elementos Importantes de las SVR:**
* Método robusto, con una base matemática muy fuerte
* Aborda el idea de hiperplanos de separación con margen máximo
* Usa elementos del dataset como vectores de soporte para maximizar el margen
* Brinda buena generalización y mitiga los problemas del overfitting
* Utiliza multiplicadores de Lagrange para el problema de optimización con restricciones
* Usa funciones kernel para problemas linealmente no separables

Creamos el modelo

In [ ]:
from sklearn.svm import SVR 

In [ ]:
svr_model = SVR(kernel='rbf')

Realizamos el entrenamiento (Ajuste de parámetros)

In [ ]:
svr_model.fit(X_train, y_train)

SVR()

Evaluamos el desempeño

In [ ]:
R2 = svr_model.score(X_test,y_test)

y_test_predict = svr_model.predict(X_test)

RMSE = (np.sqrt(mean_squared_error(y_test, y_test_predict)))

print("coeficiente de determinación R^2: ", R2)
print('----------------------------------------')
print('RMSE (root mean square error) nos da la diferencia entre los resultados reales y nuestros resultados calculados del modelo:')
print('Rmse ',RMSE)

coeficiente de determinación R^2:  0.09863695871308265
----------------------------------------
RMSE (root mean square error) nos da la diferencia entre los resultados reales y nuestros resultados calculados del modelo:
Rmse  59.88710816895359
